In [1]:
# This pylab is just to make things easier. Though generally its better to import matplotlib/numpy by themselves
%pylab inline

# Import LALSuite stuff
import lal
import lalsimulation as lalsim

import warnings
warnings.filterwarnings('ignore')

# Append path to fisher matrix code
import sys
sys.path.insert(0,'./fisher_matrix/')

# Import fisher stuff
import utils as ut
import waveform as wfm
import fisher as fsh

# These imports generally might not be needed. Could be commented out 
import pycbc as py
from pycbc.filter import match
import bilby
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
# This was a additon I needed to do to get pymultinest to work. Might not be needed for others

import os

#os.environ['DYLD_LIBRARY_PATH'] = '/Users/chinmay/gits/MultiNest/lib'


In [3]:
# Set the param_limits dictionary.
param_limits = {
    "mchirp": [0.5, 100.] ,
    "eta" : [0.01, 0.25] , 
    "spin1x" : [-1, 1.] , 
    "spin1y": [-1, 1.] , 
    "spin1z": [-1, 1.] , 
    "spin2x": [-1, 1.] , 
    "spin2y": [-1, 1.] , 
    "spin2z": [-1, 1.] , 
    "inc": [-np.pi, np.pi], 
    "ra": [0, 2*np.pi], 
    "dec": [-np.pi, np.pi], 
    "distance": [0., 1e7*1e6], 
    "phi0" : [0., 2*np.pi], 
    "psi" : [0., 2*np.pi], 
    "tgps" : [0., 1e12],
    "tcoal" : [0., 1e12]}




In [4]:
# This will be the dictionary of input parameters. Can be anything you want. 
paramsdic = {'mchirp': 2.2,
 'eta': 0.22,
 'spin1x': 0.0,
 'spin1y': 0.0,
 'spin1z': 0.1,
 'spin2x': 0.0,
 'spin2y': 0.0,
 'spin2z': 0.1,
 'inc': 0.4,
 'ra': 0.8,
 'dec': 0.2,
 'distance': 100.0,
 'phi0': 1.0,
 'fref': 5.0,
 'flow': 5.0,
 'fhigh': 1024.0,
 'psi': 0.0,
 'tgps': 1126259462.0,
 'df': 0.5,
 'tcoal': 0.2}

In [5]:
# Define parameters required for fisher computation


params_met = ["mchirp", "eta", "spin1z", "spin2z", "ra", "dec", "distance", "phi0", "inc", "tgps",  "tcoal"]

In [6]:
approx='IMRPhenomXPHM'

# Define fisher matrix individually for each detector
met_h1 = fsh.FisherMatrix_SingleIFO(paramsdic, params_met, param_limits, 'H1', approx)
met_l1 = fsh.FisherMatrix_SingleIFO(paramsdic, params_met, param_limits, 'L1', approx)
met_v1 = fsh.FisherMatrix_SingleIFO(paramsdic, params_met, param_limits, 'V1', approx)


In [7]:
fish_full = met_h1._fisher_matrix(paramsdic, params_met) + met_l1._fisher_matrix(paramsdic, params_met) + met_v1._fisher_matrix(paramsdic, params_met) 

In [8]:
#cov_mat = inv(fish_full)

cov_mat = pinv(fish_full, rcond=1e-60)

In [10]:
np.dot(cov_mat, fish_full)

array([[ 1.00000000e+00, -2.42284500e-11, -1.89387220e-11,
        -4.33698390e-12, -5.85931680e-13, -4.53206274e-14,
         4.02050030e-16,  3.14117637e-15,  5.02774023e-15,
         1.84301795e-17, -5.06397728e-12],
       [-3.62283874e-12,  1.00000000e+00,  7.28815249e-14,
         2.25756040e-14,  6.75000867e-16, -8.36234503e-17,
        -1.11803343e-18, -4.08632454e-17, -1.07587666e-17,
         2.08663992e-19, -1.14322821e-13],
       [-1.83683279e-06,  6.04014056e-08,  1.00000004e+00,
         6.39896485e-09,  8.53701624e-10,  8.42998543e-10,
        -5.54724702e-13,  3.13156265e-11, -5.17032326e-12,
         1.97648159e-13, -2.72864710e-08],
       [ 2.65230702e-06, -1.17120231e-07, -5.80036600e-08,
         1.00000000e+00,  1.13794183e-09, -4.04510742e-10,
         1.31953289e-12,  1.27571890e-10,  1.05901728e-11,
        -2.43771801e-13, -4.73688999e-09],
       [ 5.88435800e-01, -3.56860291e-03, -7.20276632e-03,
        -2.84925052e-03,  1.00003177e+00, -1.40741295e-04,
  